In [1]:
from helper import ChecklistTestRunner
runner = ChecklistTestRunner(model_path='../../Models/SquadFinetuned')


In [2]:

runner.run_suite_on_jsonl(jsonl_file='ChecklistData/checklist_test.jsonl', output_folder='checklist_results', save_results=True)

🔄 Processing ChecklistData/checklist_test.jsonl...
📊 Found 16 test groups in ChecklistData/checklist_test.jsonl
  - Comparisons: 239 testcases, 478 examples
  - Intensifiers: 239 testcases, 2868 examples
  - Properites: 240 testcases, 960 examples
  - Profession vs nationality: 240 testcases, 2400 examples
  - Animal vs Vehicle: 240 testcases, 960 examples
  - Animal vs Vehicle v2: 238 testcases, 952 examples
  - Synonyms: 217 testcases, 868 examples
  - Antonyms: 238 testcases, 952 examples
  - Antonyms Comparison: 239 testcases, 3824 examples
  - Change in profession: 231 testcases, 462 examples
  - Time Difference: 239 testcases, 956 examples
  - Negation in context, may or may not be in question: 239 testcases, 956 examples
  - Negation in question only.: 232 testcases, 1856 examples
  - Basic coref, he / she: 233 testcases, 1864 examples
  - Basic coref, his / her: 240 testcases, 480 examples
  - Former / Latter: 230 testcases, 920 examples
🧪 Running tests with model predictions..

| Test Name                                          |   Total Cases |   Example Per Case |   Failures | Failure Rate   | Average Failure   |
|----------------------------------------------------|---------------|--------------------|------------|----------------|-------------------|
| Comparisons                                        |           239 |                  2 |        239 | 100.00%        | 50.00%            |
| Intensifiers                                       |           239 |                 12 |        239 | 100.00%        | 48.36%            |
| Properites                                         |           240 |                  4 |        240 | 100.00%        | 51.25%            |
| Profession vs nationality                          |           240 |                 10 |        118 | 49.17%         | 7.79%             |
| Animal vs Vehicle                                  |           240 |                  4 |        239 | 99.58%         | 55.52%            |
| Animal vs Vehicle v2                               |           238 |                  4 |        113 | 47.48%         | 14.50%            |
| Synonyms                                           |           217 |                  4 |         16 | 7.37%          | 1.84%             |
| Antonyms                                           |           238 |                  4 |        238 | 100.00%        | 49.79%            |
| Antonyms Comparison                                |           239 |                 16 |        239 | 100.00%        | 50.05%            |
| Change in profession                               |           231 |                  2 |          2 | 0.87%          | 0.43%             |
| Time Difference                                    |           239 |                  4 |        239 | 100.00%        | 53.77%            |
| Negation in context, may or may not be in question |           239 |                  4 |        235 | 98.33%         | 24.58%            |
| Negation in question only.                         |           232 |                  8 |        232 | 100.00%        | 50.05%            |
| Basic coref, he / she                              |           233 |                  8 |        233 | 100.00%        | 99.46%            |
| Basic coref, his / her                             |           240 |                  2 |        240 | 100.00%        | 95.83%            |
| Former / Latter                                    |           230 |                  4 |        230 | 100.00%        | 100.00%           |

💾 Saving results to checklist_results/...
✅ Saved, 21756 rows to checklist_results/detailed_results.jsonl.
🔒 Generated 21756 unique IDs (collision-free)
✅ All results saved in folder: checklist_results
